Notice that:
- best start: set m = 5000 and n = 5 in _numeric.py
- random start: set m = 1 and n = 1 instead

In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from caviar import CaviarModel
from var_tests import binomial_test, kupiec_pof_test, christoffersen_test, traffic_light_test

In [3]:
while True:
    data = yf.download('SPY', start='2001-01-01')
    if data.shape[0] > 0:
        break

[*********************100%***********************]  1 of 1 completed


In [4]:
log_returns = (data['Close'].apply(np.log).diff() * 100).dropna()
in_samples = log_returns[:'2015']
out_samples = log_returns['2016':]

In [5]:
hit_rate = lambda ret, var: np.mean(ret < var)

In [6]:
quantiles = [.05, .01]
methods = ['numeric'] # ['numeric', 'mle']
models = ['adaptive', 'symmetric', 'asymmetric', 'igarch']

In [7]:
stats_df = []
beta_stats_df = []

for q in quantiles:
    for model in models:
        for method in methods:
            print(f'Runing quantile: {q}, model: {model}, method: {method}')
            for i in range(30):
                caviar_model = CaviarModel(q, model, method)
                caviar_model.fit(in_samples)
                
                # as the last observation is the VaR forecast
                in_VaR = caviar_model.predict(in_samples, caviar_model.VaR0_in)[:-1]
                out_VaR = caviar_model.predict(out_samples, caviar_model.VaR0_out)[:-1]
                
                # statistics
                stat = {
                    'quantile': q,
                    'model': model,
                    'method': method,
                    'trial': i,
                    'loss': caviar_model.training_loss,
                    'hit_rate_in': hit_rate(in_samples, in_VaR),
                    'hit_rate_out': hit_rate(out_samples, out_VaR),
                    'dq_in': caviar_model.dq_test(in_samples, 'in'),
                    'dq_out': caviar_model.dq_test(out_samples, 'out'),
                    'binom_in': binomial_test(in_samples, in_VaR, q),
                    'binom_out': binomial_test(out_samples, out_VaR, q),
                    'traffic_in': traffic_light_test(in_samples, in_VaR, q)[0],
                    'traffic_out': traffic_light_test(out_samples, out_VaR, q)[0],
                    'kupiec_in': kupiec_pof_test(in_samples, in_VaR, q),
                    'kupiec_out': kupiec_pof_test(out_samples, out_VaR, q),
                    'independent_in': christoffersen_test(in_samples, in_VaR),
                    'independent_out': christoffersen_test(out_samples, out_VaR)
                }
                
                beta_stat = caviar_model.beta_summary()
                beta_stat['quantile'] = q
                beta_stat['model'] = model
                beta_stat['method'] = method
                beta_stat['trial'] = i
                
                stats_df.append(stat)
                beta_stats_df.append(beta_stat)

Runing quantile: 0.05, model: adaptive, method: numeric
Generating 5 best initial betas out of 5000...
Optimizing...
when m = 1
Update 0: 0.14858170514362934
Update 1: 0.12817363708634252
Update 2: 0.12817363708634252
when m = 2
Update 0: 0.1486757005816056
Update 1: 0.14811025677473186
Update 2: 0.14811025677473186
when m = 3
Update 0: 0.14900041400076827
Update 1: 0.14811025588240564
Update 2: 0.12817363712979118
Update 3: 0.12817363712979118
when m = 4
Update 0: 0.15235528237854953
Update 1: 0.12817363708963456
Update 2: 0.12817363708963456
when m = 5
Update 0: 0.15434848714988625
Update 1: 0.12817363709070925
Update 2: 0.12817363709070925
Final loss: 0.12817363708634252
Time taken(s): 31.77
Generating 5 best initial betas out of 5000...


..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)
..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)


Optimizing...
when m = 1
Update 0: 0.1483832267055984
Update 1: 0.14811025677472922
Update 2: 0.14811025677472922
when m = 2
Update 0: 0.149877817877682
Update 1: 0.14811025677475306
Update 2: 0.1481102567747196
when m = 3
Update 0: 0.15039477078228555
Update 1: 0.1481102567747444
Update 2: 0.1481102567747444
when m = 4
Update 0: 0.15043845598031344
Update 1: 0.14811025677473164
Update 2: 0.14811025677473164
when m = 5
Update 0: 0.15179139903659752
Update 1: 0.14811025488277835
Update 2: 0.12817363712842858
Update 3: 0.12817363712842858
Final loss: 0.12817363712842858
Time taken(s): 31.67


..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)
..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)


Generating 5 best initial betas out of 5000...
Optimizing...
when m = 1
Update 0: 0.14811055450155625
Update 1: 0.14811025674850262
Update 2: 0.128173637221313
Update 3: 0.128173637221313
when m = 2
Update 0: 0.15231737979659415
Update 1: 0.14811025357708713
Update 2: 0.1281736370869384
Update 3: 0.1281736370869384
when m = 3
Update 0: 0.15296328755016894
Update 1: 0.14811025677472062
Update 2: 0.14811025677472062
when m = 4
Update 0: 0.15492167808988103
Update 1: 0.12817363708772786
Update 2: 0.12817363708772786
when m = 5
Update 0: 0.15495817462948658
Update 1: 0.128173637089804
Update 2: 0.128173637089804
Final loss: 0.1281736370869384
Time taken(s): 30.36
Generating 5 best initial betas out of 5000...


...

..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)
..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)


Optimizing...
when m = 1
Update 0: 0.03428156395062308
Update 1: 0.03421516824836481
Update 2: 0.034215168158223584
when m = 2
Update 0: 0.03429158152871175
Update 1: 0.03427574974359305
Update 2: 0.034275749738630096
when m = 3
Update 0: 0.03432921561424271
Update 1: 0.0342155489078291
Update 2: 0.034215530776155696
Update 3: 0.03421553075262407
when m = 4
Update 0: 0.03433156460655381
Update 1: 0.03421976463406065
Update 2: 0.03421923105919486
Update 3: 0.034216673407235706
Update 4: 0.034216672529103924
Update 5: 0.034216672529103924
when m = 5
Update 0: 0.034347263178065184
Update 1: 0.034214644829334964
Update 2: 0.03421464134436684
Update 3: 0.034214641341587096
Final loss: 0.034214641341587096
Time taken(s): 16.71
Generating 5 best initial betas out of 5000...


..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)
..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)


Optimizing...
when m = 1
Update 0: 0.03424028292413441
Update 1: 0.034218498389916086
Update 2: 0.0342184983726382
when m = 2
Update 0: 0.034243549085002135
Update 1: 0.03423318350940031
Update 2: 0.03423318350915127
when m = 3
Update 0: 0.03424726701385587
Update 1: 0.03422723992262145
Update 2: 0.0342272398864125
when m = 4
Update 0: 0.034264335278000005
Update 1: 0.034216071194703886
Update 2: 0.034216071194703886
when m = 5
Update 0: 0.034281498015563655
Update 1: 0.03423164012352673
Update 2: 0.03423163933632916
Update 3: 0.03423163932948472
Final loss: 0.034216071194703886
Time taken(s): 15.91


..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)
..\var_tests\_test.py:75: RuntimeWarning: divide by zero encountered in log
  LR_POF = -2 * np.log(numerator / denominator)


In [8]:
pd.DataFrame(stats_df)

,quantile,model,method,trial,loss,hit_rate_in,hit_rate_out,dq_in,dq_out,binom_in,binom_out,traffic_in,traffic_out,kupiec_in,kupiec_out,independent_in,independent_out
0,0.05,adaptive,numeric,0,0.128174,0.048515,0.044956,0.618006,0.998807,0.708729,0.360765,green,green,1.0,1.0,0.507619,0.960971
1,0.05,adaptive,numeric,1,0.128174,0.048515,0.044956,0.617992,0.998807,0.708729,0.360765,green,green,1.0,1.0,0.507619,0.960971
2,0.05,adaptive,numeric,2,0.128174,0.048515,0.044956,0.618005,0.998807,0.708729,0.360765,green,green,1.0,1.0,0.507619,0.960971
3,0.05,adaptive,numeric,3,0.128174,0.048515,0.044956,0.618005,0.998807,0.708729,0.360765,green,green,1.0,1.0,0.507619,0.960971
4,0.05,adaptive,numeric,4,0.128174,0.048515,0.044956,0.618006,0.998807,0.708729,0.360765,green,green,1.0,1.0,0.507619,0.960971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.01,igarch,numeric,25,0.034215,0.010339,0.013158,1.000000,0.992330,0.805834,0.192984,green,green,1.0,1.0,0.992732,0.676898
236,0.01,igarch,numeric,26,0.034215,0.010339,0.013158,1.000000,0.992330,0.805834,0.192984,green,green,1.0,1.0,0.992732,0.676898
237,0.01,igarch,numeric,27,0.034215,0.010339,0.013158,1.000000,0.992330,0.805834,0.192984,green,green,1.0,1.0,0.992732,0.676898
238,0.01,igarch,numeric,28,0.034215,0.010074,0.013158,1.000000,0.992330,0.934696,0.192984,green,green,1.0,1.0,0.993802,0.676898


In [9]:
stats_df = pd.DataFrame(stats_df)
beta_stats_df = pd.concat(beta_stats_df)
beta_stats_df.index.name = 'beta'

In [10]:
stats_df.to_csv('stat_RQ_best_start.csv', index=None)
beta_stats_df.to_csv('beta_RQ_best_start.csv')